In [1]:
import transformers
from datasets import load_dataset, load_metric

In [2]:
max_input_length = 1024

In [ ]:
schemapile_dataset = load_dataset("../../data/foreign_keys_instruction_data_schemapile")

In [4]:
import nltk
import string
from transformers import AutoTokenizer

model_checkpoint = "t5-base"
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint, model_max_length=max_input_length)

In [ ]:
schemapile_dataset_cleaned = schemapile_dataset.filter(
    lambda example: (len(example['messages'][0]['content']) < max_input_length)
)

In [9]:
max_target_length = 512
def preprocess_data(examples):
    inputs = [message[0]['content'] for message in examples['messages']]
    model_inputs = tokenizer(inputs, max_length=max_input_length, truncation=True)

    # Setup the tokenizer for targets
    with tokenizer.as_target_tokenizer():
        outputs = [message[1]['content'] for message in examples['messages']]
        labels = tokenizer(outputs, max_length=max_target_length, 
                         truncation=True)

    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

In [ ]:
tokenized_datasets = schemapile_dataset_cleaned.map(preprocess_data, batched=True)

In [11]:
from transformers import AutoModelForSeq2SeqLM, DataCollatorForSeq2Seq,  Seq2SeqTrainingArguments, Seq2SeqTrainer

In [12]:
import os
os.environ["WANDB_PROJECT"] = "t5-base-schemapile"  # name your W&B project
os.environ["WANDB_LOG_MODEL"] = "checkpoint"  # log all model checkpoints

In [13]:
batch_size = 16
eval_batch_size = 64
model_name = "t5-base-schemapile"
model_dir = f"../../{model_name}"

args = Seq2SeqTrainingArguments(
    model_dir,
    evaluation_strategy="no",
    eval_steps=1000,
    logging_strategy="steps",
    logging_steps=1000,
    save_strategy="steps",
    save_steps=1000,
    learning_rate=4e-5,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=eval_batch_size,
    weight_decay=0.01,
    save_total_limit=20,
    num_train_epochs=1,
    predict_with_generate=True,
    fp16=True,
    metric_for_best_model="loss",
    report_to="wandb"
)

In [14]:
data_collator = DataCollatorForSeq2Seq(tokenizer)

In [ ]:
metric = load_metric("rouge")

In [16]:
import numpy as np

def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    decoded_preds = tokenizer.batch_decode(predictions, skip_special_tokens=True)
    
    # Replace -100 in the labels as we can't decode them.
    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)
    
    # Compute ROUGE scores
    result = metric.compute(predictions=decoded_preds, references=decoded_labels,
                            use_stemmer=True)

    # Extract ROUGE f1 scores
    result = {key: value.mid.fmeasure * 100 for key, value in result.items()}
    
    # Add mean generated length to metrics
    prediction_lens = [np.count_nonzero(pred != tokenizer.pad_token_id)
                      for pred in predictions]
    result["gen_len"] = np.mean(prediction_lens)
    
    return {k: round(v, 4) for k, v in result.items()}

In [17]:
# Function that returns an untrained model to be trained
def model_init():
    return AutoModelForSeq2SeqLM.from_pretrained(model_checkpoint)

trainer = Seq2SeqTrainer(
    model_init=model_init,
    args=args,
    train_dataset=tokenized_datasets["train"],
    data_collator=data_collator,
    tokenizer=tokenizer,
)

In [ ]:
trainer.train()

In [19]:
trainer.save_model(model_dir)